In [1]:
# importing libraries for data preprocessing
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#importing libraries for neural network model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

# import libraries for plotting
import cufflinks as cf
import plotly.offline
from  plotly.offline import plot_mpl
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# import libraries for error metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


/home/cooldudeanmol/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# data preprocessing
# reading dataset
data = pd.read_csv("household_power_consumption.csv")
data.head()

/home/cooldudeanmol/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning:

Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.



,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16-12-2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16-12-2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16-12-2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16-12-2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16-12-2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [3]:
# making column datetime
data['Date'] = pd.to_datetime(data['Date'], format = "%d-%m-%Y" )
data = data.assign(Date=data.Date.dt.round('H'))
data = data.reset_index().drop_duplicates(subset='Date', keep='last').set_index('Date')
data.head()

,index,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date,,,,,,,,,
2006-12-16,395,23:59:00,2.586,0.070,241.630,11.000,0.000,1.000,0.0
2006-12-17,1835,23:59:00,0.276,0.120,244.890,1.200,0.000,1.000,0.0
2006-12-18,3275,23:59:00,0.434,0.248,241.270,2.000,0.000,2.000,0.0
2006-12-19,4715,23:59:00,0.448,0.052,245.680,1.800,0.000,0.000,0.0
2006-12-20,6155,23:59:00,1.888,0.220,243.050,7.800,0.000,0.000,18.0


In [4]:
# removing unwanted data
data = data['Global_active_power']

# changing column name
data.columns = ['Energy Production']
data = data[data != '?']
data = data.astype('float64')

data.head()


Date
2006-12-16    2.586
2006-12-17    0.276
2006-12-18    0.434
2006-12-19    0.448
2006-12-20    1.888
Name: Global_active_power, dtype: float64

In [5]:
print('Min', np.min(data))
print('Max', np.max(data))

Min 0.08
Max 5.698


In [6]:
# normalizing the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(np.array(data).reshape(-1, 1))

In [7]:
#plotting the data 
data.iplot(title="power", xTitle='TIme Stamp',yTitle='Demand')

In [8]:
print('Min', np.min(scaled))
print('Max', np.max(scaled))

Min 0.0
Max 1.0


In [9]:
len(scaled)
# !pip3 install pyramid-arima


1427

In [10]:
#create train/test datasets

train_size = int(len(scaled)*.7)
test_size = int(len(scaled - train_size))
train, test = scaled[:train_size, :], scaled[train_size:, :]

print(len(train), len(test))

998 429


In [11]:
#changing data into required window format  
def create_dataset(dataset, look_back = 1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[(i+look_back), 0])
        
    return np.array(dataX), np.array(dataY)

In [12]:
look_back = 12
Xtrain, Ytrain = create_dataset(train, look_back)
Xtest, Ytest = create_dataset(test, look_back)

In [13]:
Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], 1))
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], 1))
print(Xtrain.shape)
print(Xtest.shape)

(985, 12, 1)
(416, 12, 1)


In [14]:
# Neural network model
batch_size = 1

# model = Sequential()
# layers = [1, 75, 100, prediction_steps]
# model.add(LSTM(layers[1], input_shape=(None, layers[0]), return_sequences=True))  # add first layer
# model.add(Dropout(0.2))  # add dropout for first layer
# model.add(LSTM(layers[2], return_sequences=False))  # add second layer
# model.add(Dropout(0.2))  # add dropout for second layer
# model.add(Dense(layers[3]))  # add output layer
# model.add(Activation('linear'))  # output layer with linear activation
    
    
# model = Sequential()
# model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=1))
# model.add(Dense(1))
# model.compile(loss = 'mean_squared_error', optimizer='adam')
# model.fit(Xtrain, Ytrain, epochs=100, batch_size=batch_size, verbose = 2, shuffle=True)

model = Sequential()
model.add(LSTM(75, input_shape=(None, 1), return_sequences=True))  # add first layer
model.add(Dropout(0.2))  # add dropout for first layer
model.add(LSTM(100, return_sequences=False))  # add second layer
model.add(Dropout(0.2))  # add dropout for second layer
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer='adam')
model.fit(Xtrain, Ytrain, epochs=50, batch_size=batch_size, verbose = 2, shuffle=True)





Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 11s - loss: 0.0231
Epoch 2/50
 - 10s - loss: 0.0225
Epoch 3/50
 - 10s - loss: 0.0224
Epoch 4/50
 - 11s - loss: 0.0221
Epoch 5/50
 - 10s - loss: 0.0220
Epoch 6/50
 - 10s - loss: 0.0222
Epoch 7/50
 - 10s - loss: 0.0221
Epoch 8/50
 - 10s - loss: 0.0220
Epoch 9/50
 - 10s - loss: 0.0221
Epoch 10/50
 - 10s - loss: 0.0218
Epoch 11/50
 - 11s - loss: 0.0218
Epoch 12/50
 - 12s - loss: 0.0217
Epoch 13/50
 - 12s - loss: 0.0220
Epoch 14/50
 - 12s - loss: 0.0220
Epoch 15/50
 - 11s - loss: 0.0216
Epoch 16/50
 - 11s - loss: 0.0218
Epoch 17/50
 - 12s - loss: 0.0217
Epoch 18/50
 - 12s - loss: 0.0216
Epoch 19/50
 - 11s - loss: 0.0216
Epoch 20/50
 - 11s - loss: 0.0216
Epoch 21/50
 - 11s - loss: 0.0218
Epoch 22/50
 - 12s - loss: 0.0215
Epoch 23/50
 - 14s - loss:

In [15]:
# make predictions 

trainPredict = model.predict(Xtrain, batch_size = batch_size)
testPredict = model.predict(Xtest, batch_size = batch_size)

trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)

ytrain = scaler.inverse_transform([Ytrain])
ytest = scaler.inverse_transform([Ytest])
    
trainScore = sqrt(mean_squared_error(ytrain[0], trainPredict[:,0]))    
print("train Score", trainScore)

testScore = sqrt(mean_squared_error(ytest[0], testPredict[:,0]))    
print("train Score", testScore)
print(ytest[0])

train Score 0.7932814405182911
train Score 0.6526494332053032
[0.748 0.594 1.322 0.236 0.21  0.222 0.198 0.348 0.39  0.402 0.568 0.354
 0.266 0.404 0.412 0.394 0.328 0.488 0.294 0.484 0.366 0.372 1.934 0.43
 0.348 0.378 0.302 0.312 0.332 2.808 0.344 0.292 0.29  0.466 2.614 0.222
 0.292 1.428 1.386 0.35  0.24  0.228 0.388 0.5   1.432 0.346 0.406 1.276
 0.37  0.322 2.    0.472 0.252 0.314 1.462 0.324 0.834 2.272 0.282 0.36
 0.344 1.492 0.42  0.274 0.658 0.296 0.42  0.848 0.796 0.308 0.408 2.272
 0.35  0.376 0.304 0.326 0.318 0.506 1.498 0.258 0.502 0.246 0.264 0.428
 0.376 0.292 0.258 0.442 1.43  0.306 2.796 1.478 1.468 0.454 0.342 0.44
 0.344 1.792 0.486 0.508 0.31  0.422 0.314 0.452 0.43  0.354 1.782 0.254
 0.25  0.304 0.394 1.45  1.518 1.4   0.388 0.428 1.702 2.578 1.39  0.328
 1.726 1.606 1.58  1.516 1.468 0.288 0.324 1.82  1.638 1.362 1.614 3.642
 1.538 0.426 1.538 1.592 0.35  0.256 0.25  1.562 1.448 0.332 0.414 0.43
 0.258 1.53  0.274 1.532 0.25  0.372 0.256 0.776 1.448 0.542 0.392

In [16]:
trainPredictPlot = np.empty_like(scaled)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = np.empty_like(scaled)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(scaled)-1, :] = testPredict




In [17]:
#plot the two values
testPredictPlot = pd.DataFrame(testPredict,index = data[len(trainPredict)+(look_back*2)+1:len(scaled)-1:].index,columns=['Prediction'])
pd.concat([data[len(trainPredict)+(look_back*2)+1:len(scaled)-1],testPredictPlot],axis=1).iplot(title="Prediction",xTitle='TIme Stamp',yTitle='Demand')

In [18]:
# plotting whole dataset with predictions
pd.concat([data,testPredictPlot],axis=1).iplot()


In [19]:
test_samp = Xtest[0:1]
Fulltest = np.array([])

x = model.predict(test_samp)
x[0]

array([0.15455599], dtype=float32)

In [20]:
#  printing r2 score of our model 
rms = sqrt(mean_squared_error(testPredict, ytest[0]))
print(rms)

0.6526494332053032


In [21]:
#  printing mean square error of our model 
r2 = r2_score(testPredict,ytest[0])
r2

-13.052679393766056

In [22]:
# while len(Fulltest) < len(testPredict):
#     x = model.predict(test_samp)
#     np.concatenate(Fulltest.reshape(1, -1), np.array(x[0][0]).reshape(1, -1));
#     test_samp[0] = test_samp[0][1:]
#     test_samp[0] = test_samp[0].concatenate(np.array(x[0][0]))
